This notebook can be run on Google Colab.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/ZKC19940412/water_ice_nep/colab-examples/example-5.ipynb)

In Colab, you can enable the GPU acceleration from `Edit` > `Notebook Settings` > `Hardware accelerator` > `GPU`.

# Install TDMDpy from Source ($\sim$ 2 min)

In [ ]:
%%capture
%cd /content/
# Fetch tdmdpy repo
! git clone https://github.com/ZKC19940412/tdmdpy.git

# Install tdmdpy from source
%cd tdmdpy
! pip install .

# Install [GPUMD](https://github.com/brucefan1983/GPUMD) from Source ($\sim$ 2 min)
- More instructions can be found : https://gpumd.org/installation.html

In [ ]:
%%capture
%cd /content/
! git clone https://github.com/brucefan1983/GPUMD.git
%cd /content/GPUMD/src/
! make -j 8
! echo "GPUMD installation finishes!"
%cd /content/

# Clean up Workspace and Fetch Nessary Files

In [ ]:
%cd /content/
! git clone https://github.com/ZKC19940412/water_ice_nep.git
! rm -r sample_data

# Import Necessary Packages

In [ ]:
from ase.io import read
import mdtraj as mdt
import numpy as np
from pynvml import *
from pylab import *
import scipy.constants as spc
from tdmdpy.atom_manipulate import decompose_dump_xyz
from tdmdpy.atom_manipulate import load_with_cell
from tdmdpy.create_systems import generate_ice_structures
from tdmdpy.create_systems import generate_water_box
from tdmdpy.thermodynamic_properties import get_block_average_quantities

# Custom Function Define

In [ ]:
def set_fig_properties(ax_list):
    tl = 6
    tw = 2
    tlm = 4

    for ax in ax_list:
        ax.tick_params(which='major', length=tl, width=tw)
        ax.tick_params(which='minor', length=tlm, width=tw)
        ax.tick_params(which='both', axis='both',
                       direction='in', right=True, top=True,
                       left=True)

# Obtain Information about GPU Architecture for Particular colab Instance

In [ ]:
if __name__ == "__main__":

    # Initialize nvml object
    nvmlInit()
    print("Driver Version:", nvmlSystemGetDriverVersion())
    deviceCount = nvmlDeviceGetCount()

    # Loop through all avaliable devices
    for i in range(deviceCount):
        handle = nvmlDeviceGetHandleByIndex(i)
        print("Device", i, ":", nvmlDeviceGetName(handle))

# Perform Simulation for Water

## Create Liquid Water

In [ ]:
if __name__ == "__main__":
    generate_water_box(target_density = 0.994,
                       number_of_molecules=512,
                       is_pre_equilibrate=False)

## Compose run.in

In [ ]:
%%writefile run.in

# NEP potential for water
potential /content/water_ice_nep/nep-pre-train-model/nep.txt

# time step for 0.5 fs
time_step 0.5
minimize sd 1.0e-6 10000

# Initialize velocity at 288K
velocity 288

# Run NPT equalibration with SCR method for 288K Tini and Tend, and 100 for Tcoupling, 0 bar for pressures, and 2 Gpa for pressure coupling and 1000 steps
ensemble  npt_scr 288 288 100 0 0 0 2 2 2 1000

# run 8000 steps, equal to 4 ps simulation
run 8000

# Run NVT production with BDP thermo-stat for 288K Tini and Tend, and 100 for Tcoupling
ensemble nvt_bdp 288 288 100

# dump extended xyz with every 100 steps, dump force and velocity too
dump_exyz 100 1 1

# dump themodynamic quantities every 100 steps
dump_thermo 100

# Run 100000 steps, equal to 50 ps simulation
run 100000

## Peform Simualtions ($\sim$ 15 min)
- 8000 and 100000 steps used here only serve as illustration purpose.

In [ ]:
! /content/GPUMD/src/gpumd < run.in
! mv dump.xyz dump_water.xyz
! mv thermo.out thermo_water.out

# Perform Simulations for Ih Ice

## Create Hexagonal Ice (ih) System

In [ ]:
if __name__ == "__main__":
    generate_ice_structures(type_str='Ih', number_of_total_replica=27)

## Peform Simualtions ($\sim$ 15 min)
- 8000 and 100000 steps used here only serve as illustration purpose.

In [ ]:
! /content/GPUMD/src/gpumd < run.in
! mv dump.xyz dump_ice.xyz
! mv thermo.out thermo_ice.out

## Visualize T vs Time from Production Run

In [ ]:
if __name__ == "__main__":

    #  Set up Figure Styles
    aw = 2
    fs = 22
    font = {'size': fs}
    matplotlib.rc('font', **font)
    matplotlib.rc('axes', linewidth=aw)

     # Declare data path
    water_run_data_path = 'thermo_water.out'
    ice_run_data_path = 'thermo_ice.out'

    # Load in data
    water_run_data = np.loadtxt(water_run_data_path)
    ice_run_data = np.loadtxt(ice_run_data_path)

    # Denote time step and sample rate
    time_step = 5e-4
    sample_rate = 100

    # Derive time span
    time_span = time_step * sample_rate * np.arange(0, len(water_run_data), 1)

    # Load temperature and compute its block averages
    temperature_water = water_run_data[:, 0]
    temperature_ice = ice_run_data[:, 0]
    block_average_temperature_water = get_block_average_quantities(temperature_water,
                                                             n_block=5)
    block_average_temperature_ice = get_block_average_quantities(temperature_ice,
                                                             n_block=5)

    print('Inspect Properties from NVT Simulations: ')
    print('\n')
    figure(figsize=(8, 6))
    set_fig_properties([gca()])
    plot(time_span,temperature_water, label='$T_{water}$')
    plot(time_span, np.round(
        block_average_temperature_water.mean(), 3) * np.ones_like(
        time_span), 'r--',
         label=r'$T_{water \ block \ avg}$', lw=3)
    xlabel('Time (ps)')
    ylabel('$T_{water}$ (K)')
    ylim([260, 320])
    legend(loc=4, fontsize=16)
    show()

    print('\n')

    figure(figsize=(8, 6))
    set_fig_properties([gca()])
    plot(time_span,temperature_water, label='$T_{ice}$')
    plot(time_span, np.round(
        block_average_temperature_water.mean(), 3) * np.ones_like(
        time_span), 'r--',  label=r'$T_{ice \ block \ avg}$', lw=3)
    xlabel('Time (ps)')
    ylabel('$T_{ice}$ (K)')
    ylim([260, 320])
    legend(loc=4, fontsize=16)
    show()

# Compute $\Delta H_{melt}$

In [ ]:
if __name__ == "__main__":

    # Denote unit conversion factor
    unit_conversion_factor_eV_to_kcal_per_mole = spc.eV * spc.value('Avogadro constant') / spc.kilo / spc.calorie
    unit_conversion_factor_kJ_to_kcal_per_mole = spc.value('Avogadro constant') /spc.kilo /spc.calorie

    # Denote number of molecules in two run
    number_of_molecules_water_run = len(read('dump_water.xyz', index='0')) / 3
    number_of_molecules_ice_run = len(read('dump_ice.xyz', index = '0')) / 3


    # Extract total energy in two phase
    per_atom_energy_water_in_kcal_per_mole_arr = unit_conversion_factor_eV_to_kcal_per_mole * np.sum(
       water_run_data[:, 1:3], axis=1)/number_of_molecules_water_run
    per_atom_energy_ice_in_kcal_per_mole_arr = unit_conversion_factor_eV_to_kcal_per_mole * np.sum(
        ice_run_data[:, 1:3], axis=1)/number_of_molecules_ice_run

    # Extract volume in two phases
    per_atom_volume_water_in_meter_cubic = (spc.angstrom **3) * np.prod(
        water_run_data[:, 9:12], axis=1)/number_of_molecules_water_run
    per_atom_volume_ice_in_meter_cubic = (spc.angstrom **3) * np.prod(
         ice_run_data[:, 9:12], axis=1)/number_of_molecules_ice_run

    # Extract pressure in two phases
    pressure_water_in_pa = spc.giga * np.sum(water_run_data[:, 3:9], axis=1)
    pressure_ice_in_pa = spc.giga * np.sum(ice_run_data[:, 3:9], axis=1)

    # Constrcut PV contributions
    per_atom_pv_water_in_kcal_per_mole_arr = unit_conversion_factor_kJ_to_kcal_per_mole * pressure_water_in_pa * per_atom_volume_water_in_meter_cubic
    per_atom_pv_ice_in_kcal_per_mole_arr = unit_conversion_factor_kJ_to_kcal_per_mole * pressure_ice_in_pa * per_atom_volume_ice_in_meter_cubic

    # Extract average quantities
    per_atom_energy_water_in_kcal_per_mole = get_block_average_quantities(per_atom_energy_water_in_kcal_per_mole_arr, n_block=5)
    per_atom_energy_ice_in_kcal_per_mole = get_block_average_quantities(per_atom_energy_ice_in_kcal_per_mole_arr, n_block=5)
    per_atom_pv_water_in_kcal_per_mole = get_block_average_quantities(per_atom_pv_water_in_kcal_per_mole_arr, n_block=5)
    per_atom_pv_ice_in_kcal_per_mole = get_block_average_quantities(per_atom_pv_ice_in_kcal_per_mole_arr, n_block=5)

    # Sum over terms for deltaHmelt
    deltaHmelt = (per_atom_energy_water_in_kcal_per_mole - per_atom_energy_ice_in_kcal_per_mole) + (per_atom_pv_water_in_kcal_per_mole - per_atom_pv_ice_in_kcal_per_mole)

    print('\n')
    print('U_liquid : %.3f kcal/mol' % per_atom_energy_water_in_kcal_per_mole.mean())
    print('U_solid: %.3f kcal/mol' % per_atom_energy_ice_in_kcal_per_mole.mean())
    print('pv_liquid: %.3f kcal/mol' %  per_atom_pv_water_in_kcal_per_mole.mean())
    print('pv_solid: %.3f kcal/mol' %  per_atom_pv_ice_in_kcal_per_mole.mean())
    print('Delta Hmelt from NEP: %.2f kcal/mol' % deltaHmelt.mean())

    # Convert unit back to meV/H2O
    deltaHmelt = deltaHmelt.copy() * spc.kilo /23.06
    print('deltaH_melt: %.1f ± %.1f meV/H2O' %(deltaHmelt.mean(),(deltaHmelt.std()/len(deltaHmelt))))
